In [ ]:
import os
from langchain_groq import ChatGroq
from langchain.chat_models import init_chat_model
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import tool
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage
load_dotenv()

GROQ_API_KEY = os.getenv("GROQ_API_KEY")
from langchain.chat_models import init_chat_model

import getpass
import os

if not os.environ.get("GROQ_API_KEY"):
  os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API key for Groq: ")


llm = init_chat_model("qwen/qwen3-32b", model_provider="groq", temperature=0)



In [24]:
from langgraph.prebuilt import create_react_agent

def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

agent = create_react_agent(
    model=llm,
    tools=[get_weather],
    prompt="You are a helpful assistant"
)

# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)


{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='95d891b3-0536-4ff0-a3fd-bc04ae82a064'),
  AIMessage(content='', additional_kwargs={'reasoning_content': "Okay, the user is asking for the weather in SF. I need to figure out which function to use. The available tool is get_weather, which requires a city parameter. SF is the abbreviation for San Francisco. So I should call get_weather with the city set to San Francisco. Let me make sure that's the correct parameter. Yep, the function takes a city string. I'll format the tool call accordingly.\n", 'tool_calls': [{'id': '7t5m992cw', 'function': {'arguments': '{"city":"San Francisco"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 109, 'prompt_tokens': 158, 'total_tokens': 267, 'completion_time': 0.214759423, 'prompt_time': 0.006093257, 'queue_time': 0.047341373, 'total_time': 0.22085268}, 'model_name': 'qwen/qwen3-32b', '

In [17]:
@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b


tools = [add, multiply]

llm_with_tools = llm.bind_tools(tools)



In [18]:
query = "What is 3 * 12? Also, what is 11 + 49?"

messages = [SystemMessage(content="Analyse the query carefully and figure out which tools to use. You HAVE to use the provided tools if you can")]+[HumanMessage(query)]

ai_msg = llm_with_tools.invoke(messages)

print(ai_msg.tool_calls)

messages.append(ai_msg)

[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'pmty0he2c', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': 'p7z7s6ntm', 'type': 'tool_call'}]


In [19]:
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages



[SystemMessage(content='Analyse the query carefully and figure out which tools to use. You HAVE to use the provided tools if you can', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'reasoning_content': "Okay, let's see. The user is asking two questions here. First, they want to know 3 multiplied by 12, and second, 11 plus 49. I need to figure out which functions to use for each part.\n\nLooking at the tools provided, there's the 'multiply' function for the first part and 'add' for the second. The multiply function takes two integers, a and b, so I'll use that for 3 and 12. Then the add function takes two integers as well, so that's for 11 and 49. \n\nI should make sure the parameters are correct. For the first tool call, the name should be 'multiply' with a=3 and b=12. The second tool call is 'add' with a=11 and b=49. Let me double-check 

In [20]:
llm_with_tools.invoke(messages)

AIMessage(content='The result of 3 * 12 is **36**, and the result of 11 + 49 is **60**.', additional_kwargs={'reasoning_content': 'Okay, let\'s see. The user asked two questions: 3 multiplied by 12 and 11 plus 49. I need to use the provided functions to get the answers.\n\nFirst, for the multiplication part, the function is called "multiply" and takes two integers. The parameters a and b are 3 and 12. So I\'ll call multiply(3, 12). The response from the tool was 36. That\'s correct because 3 times 12 is indeed 36.\n\nNext, the addition part. The function here is "add" with a and b as 11 and 49. Adding those together, 11 + 49 equals 60. The tool\'s response was 60, which matches my calculation.\n\nNow, I need to present both answers clearly. The user probably wants a straightforward answer without any extra details. Just the results of both operations. I should format it in a way that\'s easy to read, maybe using bullet points or a simple sentence structure. Let me make sure there are n